In [1]:
# 載入資料

import pandas as pd
train = pd.read_csv("./titanic/train.csv", encoding="UTF8")
test = pd.read_csv("./titanic/test.csv", encoding="UTF8")

# train
# test
# 891 rows × 12 columns

In [2]:
# 找空格 in train

# 是 NaN > 帶回 TRUE，不是 NaN > 則帶回 FALSE。
NaN = train.isna()

# 統計有多少 NaN
sumNaN = NaN.sum()
# sumNaN

# 補值
# Age & Cabin & Embarked
# Cabin 實在缺太多不補

# 先算中位數 -> '類別資料' 不能補中位數 (ex. PClass = 艙等 = 補眾數)
# 沒缺的可drop也可以不drop -> 因為沒缺值沒辦法補
med = train.median()
# med
med_drop = med.drop("Pclass")
# med_drop

# 要回存train
train = train.fillna(med_drop)

# 補最常出現的 (眾數)
# Embarked = 出發地

# value_counts() = 出現幾次, idxmax() = 找最大值的index
most = train["Embarked"].value_counts().idxmax()

# 填值後記得回存
train["Embarked"] = train["Embarked"].fillna(most)

# 在看一次有無空值
countNaN_train = train.isna().sum()
# countNaN_train

In [3]:
# 找空格 in test

test_NaN = test.isna()
test_sumNaN = test_NaN.sum()
# test_sumNaN

# 補值
# Age & Cabin
# Cabin 實在缺太多不補

# 不可以再算一次 test.median()
# 要用 train.medium()補
# ^^^^^^^^^^^^^^^^^^^^^  (重點)

test = test.fillna(med_drop)

# 在看一次有無空值
countNaN_test = test.isna().sum()
# countNaN_test

In [4]:
# 處理 名子裡的關鍵字
# 正則表達式

import re

s = "Braund, Mr. Owen Harris"
s = s.split(",")[-1].split(".")[0]
s.strip()

'Mr'

In [5]:
def namecut(s):
    reserve = ["Mr", "Miss", "Mrs", "Master"]
    
    s = s.split(",")[-1].split(".")[0]
    s = s.strip()
    
    if s in reserve:
        return s 
    
    else:
        return None

# --------------------------------------------------

# value_counts() > 有些稱謂量太少 會過擬合
# 記得回存

# train["Name"].apply(namecut).value_counts()
train["Name"] = train["Name"].apply(namecut)
test["Name"] = test["Name"].apply(namecut)

# train
# test

In [6]:
# one-hot encoding
# pandas.get_dummies() = 函數功能：將類別變量轉換為標籤變量。

# ["Name"] 現在已經被改成稱謂了

x_train = pd.get_dummies(train,
                         columns = ["Name", "Sex", "Embarked"])

x_predict = pd.get_dummies(test,
                         columns = ["Name", "Sex", "Embarked"])

# x_train
# x_predict

In [7]:
# 刪掉毫無預測價值的直列
x_train2 = x_train.drop(["PassengerId", "Survived", "Ticket", "Cabin"], axis = 1)

# test中刪掉一樣的直列
x_predict2 = x_predict.drop(["PassengerId", "Ticket", "Cabin"], axis = 1)

# 答案
y_train2 = x_train["Survived"]

# 保留
predict_id = x_predict["PassengerId"]

# x_train2
# x_predict2
# y_train2

In [8]:
# 隨機森林 開始
# ps.先算預測力
# 這區是先算預測力後來拿調參數衝預測力用的

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

params = {
    "n_estimators": range(20, 110, 10),
    "max_depth": range(5, 11)
}

#----------------------------------------------------------
# 算最佳參數

Grid = GridSearchCV(clf, params, cv = 10, n_jobs = 4 )
Grid.fit(x_train2, y_train2)

print("最佳參數:", Grid.best_params_)
print("最佳平均分數:", Grid.best_score_)

最佳參數: {'max_depth': 9, 'n_estimators': 80}
最佳平均分數: 0.838426966292135


In [9]:
# 先寫這欄
# 算預測力
# ps.先算預測力

from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators = 60, max_depth = 7)

scores = cross_val_score(clf,
                         x_train2,
                         y_train2,
                         cv = 10,
                         n_jobs = 4)

import numpy as np
scores = np.around(scores, decimals=3)

print("10次分數:", list(scores))
print("")
print("average:", sum(scores)/len(scores))

10次分數: [0.767, 0.876, 0.73, 0.899, 0.899, 0.809, 0.82, 0.764, 0.865, 0.831]

average: 0.8260000000000002


In [10]:
# '真樹' 在此
# 來預測喔--->

clf = RandomForestClassifier(n_estimators = 27, max_depth = 7)
clf.fit(x_train2, y_train2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=27,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
# 來預言喔

pre = clf.predict(x_predict2)
# pre

In [12]:
# 預言要補上對應 ID

result = pd.DataFrame({
        "PassengerId":predict_id,
        "Survived":pre})

# 存檔
result.to_csv("titanic_rf.csv", encoding="UTF8", index=False)

# result